In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

In [7]:
facebook = pd.read_csv("/Users/lujiawen/Desktop/数据分析课-note/数据集/FBlocation-fb/train.csv")

In [12]:
#地点
facebook_data = facebook.query("x>2.0 & x<2.5 & y>2.0 & y<2.5")
#时间
time = pd.to_datetime(facebook_data["time"], unit="s")
time = pd.DatetimeIndex(time)  #DatetimeIndex 看看是啥 ，是不是把时间格式转换一下？？
facebook_data["day"] = time.day
facebook_data["hour"] = time.hour
facebook_data["weekday"] = time.weekday
# 2.3 去掉签到较少的地方
place_count = facebook_data.groupby("place_id").count()
place_count = place_count[place_count["row_id"]>3]


#建议赋值给一个新变量。why？？数据覆盖？？ 万一错了还有机会回头
facebook_data_new = facebook_data[facebook_data["place_id"].isin(place_count.index)] ##?????

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#index

In [13]:
# 2.4 确定特征值和目标值
x = facebook_data_new[["x", "y", "accuracy", "day", "hour", "weekday"]] #特征
y = facebook_data_new["place_id"] #目标
# 2.5 分割数据集
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=22)
# 3.1 实例化一个转换器
transfer = StandardScaler()
# 3.2 调用fit_transform
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

### 省去了训练的步骤，直接加载之前保存好的模型

In [14]:
estimator = joblib.load('/Users/lujiawen/Desktop/fb_estimator.pkl')
estimator

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 3, 5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 模型预测&评估

In [17]:
y_predict = estimator.predict(x_test)
print("最后的预测值为:\n", y_predict)
print('————————————评估————————————')
# 5.1 基本评估方式
score = estimator.score(x_test, y_test)
print("最后预测的准确率为:\n", score)
# print("预测值和真实值的对比情况:\n", y_predict == y_test)
print(sum( y_predict == y_test)/len(y_predict))
# 5.2 使用交叉验证后的评估方式
print("在交叉验证中验证的最好结果:\n", estimator.best_score_)
print("最好的参数模型:\n", estimator.best_estimator_)
print("每次交叉验证后的验证集准确率结果和训练集准确率结果:\n",estimator.cv_results_)

最后的预测值为:
 [9983648790 6329243787 9674001925 ... 2990018952 4830766946 7065571836]
————————————评估————————————
最后预测的准确率为:
 0.36515361515361516
0.36515361515361516
在交叉验证中验证的最好结果:
 0.35462385462385465
最好的参数模型:
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')
每次交叉验证后的验证集准确率结果和训练集准确率结果:
 {'mean_fit_time': array([0.06876025, 0.06801252, 0.06029029, 0.06253452, 0.05889335]), 'std_fit_time': array([0.01174866, 0.01463424, 0.00290684, 0.00371409, 0.0019377 ]), 'mean_score_time': array([0.23749218, 0.48940473, 0.51673989, 0.6071816 , 0.6778975 ]), 'std_score_time': array([0.04202111, 0.09729483, 0.01998819, 0.04778613, 0.03480236]), 'param_n_neighbors': masked_array(data=[1, 3, 5, 7, 9],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors': 1}, {'n_neighbors': 3}, {'n_neighbors': 5}, {'n_neighbors': 7}, {'